#   Flight delay Analysis and Prediction


In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train data

In [3]:
#train = pd.read_csv('Train.csv')
train = pd.read_csv('/content/drive/MyDrive/Final hackathon/Train.csv')
print(train.shape)

train.head()

(7861, 12)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46


### 1. Deriving the Target attribute FlightDelayStatus by using ActualArrivalTimeStamp

* **FlightDelayStatus**   *=* ActualArrivalTimeStamp - ScheduledArrTime 


* If the delay is more than  15 minutes then **FlightDelayStatus** will be **“1”** else **“2”**


* The Scheduled flight arrival time indicates (**15** is **0:15AM**, **215** is **2:15AM**, and **1800** is **6PM**), which is in the 24-hour clock format


### 2. Converting hours into minutes to find out the difference between ActualArrivalTimeStamp and ScheduledArrTime

#### for example 
    -> 1154 = 11 * 60 + 54 = 714 minutes

## zfill()

* The **zfill()** method adds zeros (0) at the **beginning of the string**, until it reaches the **specified length**.


* If the value of the len parameter is less than the length of the string, no filling is done.


Source : [Link](https://www.w3schools.com/python/ref_string_zfill.asp)

In [4]:
train['ScheduledArrTime_00'] = train['ScheduledArrTime'].astype(str)
train['ScheduledArrTime_00'] = train['ScheduledArrTime_00'].str.zfill(4)

In [5]:
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946


In [6]:
b = train['ScheduledArrTime_00'].values

Hour = []
for values in b :
    n = values[0:2]
    Hour.append(n)
train['ScheArrTimeHours_Val'] = Hour

In [7]:
b = train['ScheduledArrTime_00'].values
min = []
for values in b :
    n = values[2:]
    min.append(n)
train['ScheArrTimeMINS_Val'] = min

In [8]:
train['ScheArrTimeHours_Val'] = train['ScheArrTimeHours_Val'].astype(str).astype(int)
train['ScheArrTimeMINS_Val'] = train['ScheArrTimeMINS_Val'].astype(str).astype(int)

In [9]:
t = train['ScheArrTimeHours_Val']
Houttomins = []
for val in t :
    mul = val * 60
    Houttomins.append(mul)
train["ScheArrTimeHourINMins"] = Houttomins

In [10]:
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154,11,54,660
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634,16,34,960
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902,19,2,1140
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533,15,33,900
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946,9,46,540


In [11]:
train['ScheduledArrTime_IN_MINS'] = train['ScheArrTimeHourINMins'] + train['ScheArrTimeMINS_Val']
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins,ScheduledArrTime_IN_MINS
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154,11,54,660,714
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634,16,34,960,994
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902,19,2,1140,1142
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533,15,33,900,933
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946,9,46,540,586


In [12]:
nn = list(train['ActualArrivalTimeStamp'].values)
Arrhhmm = []
for val in nn :
    hh = val[9:11]
    mm = val[12:14]
    hhmm = hh+mm
    Arrhhmm.append(hhmm)
    
train['ActArrTimein_HHMM'] = Arrhhmm

In [13]:
nn = list(train['ActualArrivalTimeStamp'].values)
Atchours = []
for val in nn:
    c = val[9:11]
    Atchours.append(c)
train['ArrivalTime_Houes_Val'] = Atchours

In [14]:
nn = list(train['ActualArrivalTimeStamp'].values)
AtcMins = []
for val in nn:
    c = val[12:14]
    AtcMins.append(c)
train['ArrivalTime_Mins_Val'] = AtcMins

In [15]:
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins,ScheduledArrTime_IN_MINS,ActArrTimein_HHMM,ArrivalTime_Houes_Val,ArrivalTime_Mins_Val
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154,11,54,660,714,1154,11,54
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634,16,34,960,994,1855,18,55
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902,19,2,1140,1142,1902,19,02
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533,15,33,900,933,1533,15,33
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946,9,46,540,586,0946,09,46


In [16]:
train['ArrivalTime_Houes_Val'] = train['ArrivalTime_Houes_Val'].astype(str).astype(int) 
train['ArrivalTime_Mins_Val'] = train['ArrivalTime_Mins_Val'].astype(str).astype(int)

In [17]:
tt = train['ArrivalTime_Houes_Val']
AtcHouttomins = []
for val in tt :
    mul = val * 60
    AtcHouttomins.append(mul)
train["ActArrTimeHour_IN_Mins"] = AtcHouttomins

In [18]:
train['ActualArrTime_IN_MINS'] = train['ActArrTimeHour_IN_Mins'] + train['ArrivalTime_Mins_Val']


In [19]:
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins,ScheduledArrTime_IN_MINS,ActArrTimein_HHMM,ArrivalTime_Houes_Val,ArrivalTime_Mins_Val,ActArrTimeHour_IN_Mins,ActualArrTime_IN_MINS
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154,11,54,660,714,1154,11,54,660,714
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634,16,34,960,994,1855,18,55,1080,1135
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902,19,2,1140,1142,1902,19,2,1140,1142
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533,15,33,900,933,1533,15,33,900,933
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946,9,46,540,586,0946,9,46,540,586


### Some flights don't have the same departure days and actual arrival days.

#### for example 

    * ScheduledDepTime = 28/09/04 17:05
    * ScheduledArrTime = 28/09/04 22:44
    * ActualArrivalTime = 29/09/04 00:40



In [45]:
train[train['Diff_Act_vs_Shed_ArrTime'] < 0 ]

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins,ScheduledArrTime_IN_MINS,ActArrTimein_HHMM,ArrivalTime_Houes_Val,ArrivalTime_Mins_Val,ActArrTimeHour_IN_Mins,ActualArrTime_IN_MINS,Diff_Act_vs_Shed_ArrTime,FlightDelayStatus
2277,DEN_EWR_53179,2004,9,28,2,1705,2244,219,DEN,EWR,1605,29/09/04 00:40,2244,22,44,1320,1364,0040,0,40,0,40,-1324,1
4563,DFW_LGA_46849,2004,7,18,7,1800,2234,214,DFW,LGA,1389,19/07/04 00:29,2234,22,34,1320,1354,0029,0,29,0,29,-1325,1
5382,DFW_BDL_37996,2004,7,28,3,1731,2204,213,DFW,BDL,1471,29/07/04 01:01,2204,22,4,1320,1324,0101,1,1,60,61,-1263,1
6141,ORD_PHL_34357,2004,11,27,6,1800,2113,133,ORD,PHL,678,28/11/04 00:12,2113,21,13,1260,1273,0012,0,12,0,12,-1261,1
7756,ORD_MIA_55678,2004,11,24,3,1750,2156,186,ORD,MIA,1197,25/11/04 00:15,2156,21,56,1260,1316,0015,0,15,0,15,-1301,1


In [46]:
train['Diff_Act_vs_Shed_ArrTime'] = abs(train['ActualArrTime_IN_MINS'] - train['ScheduledArrTime_IN_MINS'])


In [44]:
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins,ScheduledArrTime_IN_MINS,ActArrTimein_HHMM,ArrivalTime_Houes_Val,ArrivalTime_Mins_Val,ActArrTimeHour_IN_Mins,ActualArrTime_IN_MINS,Diff_Act_vs_Shed_ArrTime,FlightDelayStatus
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154,11,54,660,714,1154,11,54,660,714,0,2
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634,16,34,960,994,1855,18,55,1080,1135,141,1
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902,19,2,1140,1142,1902,19,2,1140,1142,0,2
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533,15,33,900,933,1533,15,33,900,933,0,2
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946,9,46,540,586,0946,9,46,540,586,0,2


In [23]:
val = train['Diff_Act_vs_Shed_ArrTime'].values

delay = []
for mins in val:
    if mins > 15 :
        delay.append(1)
    else :
        delay.append(2)    
train['FlightDelayStatus'] = delay
train.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_00,ScheArrTimeHours_Val,ScheArrTimeMINS_Val,ScheArrTimeHourINMins,ScheduledArrTime_IN_MINS,ActArrTimein_HHMM,ArrivalTime_Houes_Val,ArrivalTime_Mins_Val,ActArrTimeHour_IN_Mins,ActualArrTime_IN_MINS,Diff_Act_vs_Shed_ArrTime,FlightDelayStatus
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,1154,11,54,660,714,1154,11,54,660,714,0,2
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1634,16,34,960,994,1855,18,55,1080,1135,141,1
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,1902,19,2,1140,1142,1902,19,2,1140,1142,0,2
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,1533,15,33,900,933,1533,15,33,900,933,0,2
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,0946,9,46,540,586,0946,9,46,540,586,0,2


 **1519**, the number of flights is **delayed**, out of **7861** flights 

In [24]:
train['FlightDelayStatus'].value_counts()

2    6342
1    1519
Name: FlightDelayStatus, dtype: int64

In [25]:
columns = ['FlightNumber', 'Year', 'Month', 'DayofMonth', 'DayOfWeek',
       'ScheduledDepTime', 'ScheduledArrTime', 'ScheduledTravelTime', 'Origin',
       'Destination', 'Distance', 'ActualArrivalTimeStamp',
      'FlightDelayStatus']

In [26]:
train1 = train[columns]
print(train1.shape)

(7861, 13)


In [27]:
train1.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2
1,CLE_ATL_44346,2004,11,19,5,1440,1634,114,CLE,ATL,554,19/11/04 18:55,1
2,SNA_LAS_44378,2004,5,4,2,1800,1902,62,SNA,LAS,226,04/05/04 19:02,2
3,CLT_TPA_51502,2004,7,6,2,1400,1533,93,CLT,TPA,508,06/07/04 15:33,2
4,MSP_ORD_44884,2004,9,23,4,830,946,76,MSP,ORD,334,23/09/04 09:46,2


# AllStationsData

In [28]:
#station = pd.read_csv('AllStationsData_PHD.txt', sep = '|')
station = pd.read_csv('/content/drive/MyDrive/Final hackathon/AllStationsData_PHD.txt', sep = '|')
print(station.shape)
station.head()

(1744, 8)


,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,3011,TEX,0,0,0,37.57,-107.55,+7
1,3012,SKX,0,0,0,36.28,-105.40,+7
2,3013,LAA,3683,3703,3675,38.04,-102.41,+7
3,3014,4SL,0,6909,0,35.47,-107.14,+7
4,3016,RIL,5503,5544,5506,39.32,-107.44,+7


### FlightDelayStatus depends on weather reports of both the Origin and Destination of the flights. 

* A flight delay is when an airline flight **takes off and/or lands** later than its scheduled time

* Flights may be delayed due to **wind, precipitation, fog or low visibility, lightning, low clouds, or storms**.


#### AllStationsData flie gives the link of **"WeatherStationID"**  and **"AirportID"**


* AllStationsData of **AirportID** has the same values as in the **Origin** and **Destination** column of the Train.


* Merging AllStationsData of **AirportID** with the Train of **Origin** column gives WeatherStationID for the **Origin of the flights**.


* Merging AllStationsData of **AirportID** with the Train of **Destination** column gives WeatherStationID for the **Destination of the flights.**

### Merging AllStationsData of AirportID with the Train of Origin

In [29]:
trainorigin = pd.merge(train1, station, left_on = 'Origin', right_on = 'AirportID')
print(trainorigin.shape)
trainorigin.head()

(7861, 21)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14739,BOS,19,30,180,42.22,-71.01,+5
1,BOS_OAK_55049,2004,9,10,5,820,1200,400,BOS,OAK,2693,10/09/04 12:00,2,14739,BOS,19,30,180,42.22,-71.01,+5
2,BOS_RSW_47586,2004,3,22,1,700,1036,216,BOS,RSW,1249,22/03/04 10:36,2,14739,BOS,19,30,180,42.22,-71.01,+5
3,BOS_PIT_49759,2004,5,1,6,845,1031,106,BOS,PIT,496,01/05/04 10:31,2,14739,BOS,19,30,180,42.22,-71.01,+5
4,BOS_DCA_32791,2004,7,15,4,1300,1430,90,BOS,DCA,399,15/07/04 14:30,2,14739,BOS,19,30,180,42.22,-71.01,+5


### Merging AllStationsData of AirportID with the Train of Destination

In [47]:
traindestination = pd.merge(train1, station, left_on = 'Destination', right_on = 'AirportID')
print(traindestination.shape)
traindestination.head()

(7861, 21)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14820,CLE,802,805,805,41.24,-81.51,+5
1,FLL_CLE_45262,2004,3,13,6,1600,1847,167,FLL,CLE,1062,13/03/04 18:47,2,14820,CLE,802,805,805,41.24,-81.51,+5
2,EWR_CLE_37064,2004,3,4,4,645,828,103,EWR,CLE,404,04/03/04 10:28,1,14820,CLE,802,805,805,41.24,-81.51,+5
3,IAD_CLE_47407,2004,9,13,1,745,900,75,IAD,CLE,288,13/09/04 09:00,2,14820,CLE,802,805,805,41.24,-81.51,+5
4,STL_CLE_54043,2004,11,7,7,1200,1433,93,STL,CLE,487,07/11/04 14:33,2,14820,CLE,802,805,805,41.24,-81.51,+5


In [48]:
trainorigin.rename(columns = {'WeatherStationID':'WeatherStationID_Origin'}, inplace = True)
trainorigin.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14739,BOS,19,30,180,42.22,-71.01,+5
1,BOS_OAK_55049,2004,9,10,5,820,1200,400,BOS,OAK,2693,10/09/04 12:00,2,14739,BOS,19,30,180,42.22,-71.01,+5
2,BOS_RSW_47586,2004,3,22,1,700,1036,216,BOS,RSW,1249,22/03/04 10:36,2,14739,BOS,19,30,180,42.22,-71.01,+5
3,BOS_PIT_49759,2004,5,1,6,845,1031,106,BOS,PIT,496,01/05/04 10:31,2,14739,BOS,19,30,180,42.22,-71.01,+5
4,BOS_DCA_32791,2004,7,15,4,1300,1430,90,BOS,DCA,399,15/07/04 14:30,2,14739,BOS,19,30,180,42.22,-71.01,+5


In [49]:
traindestination.rename(columns = {'WeatherStationID':'WeatherStationID_Destination'}, inplace = True)
traindestination.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Destination,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14820,CLE,802,805,805,41.24,-81.51,+5
1,FLL_CLE_45262,2004,3,13,6,1600,1847,167,FLL,CLE,1062,13/03/04 18:47,2,14820,CLE,802,805,805,41.24,-81.51,+5
2,EWR_CLE_37064,2004,3,4,4,645,828,103,EWR,CLE,404,04/03/04 10:28,1,14820,CLE,802,805,805,41.24,-81.51,+5
3,IAD_CLE_47407,2004,9,13,1,745,900,75,IAD,CLE,288,13/09/04 09:00,2,14820,CLE,802,805,805,41.24,-81.51,+5
4,STL_CLE_54043,2004,11,7,7,1200,1433,93,STL,CLE,487,07/11/04 14:33,2,14820,CLE,802,805,805,41.24,-81.51,+5


In [50]:
cols = ['FlightNumber', 'AirportID','WeatherStationID_Destination', 'GroundHeight','StationHeight','BarometerHeight','Latitude','Longitude','TimeZone']
Dest_data = traindestination[cols]
Dest_data.head()

,FlightNumber,AirportID,WeatherStationID_Destination,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,BOS_CLE_29089,CLE,14820,802,805,805,41.24,-81.51,+5
1,FLL_CLE_45262,CLE,14820,802,805,805,41.24,-81.51,+5
2,EWR_CLE_37064,CLE,14820,802,805,805,41.24,-81.51,+5
3,IAD_CLE_47407,CLE,14820,802,805,805,41.24,-81.51,+5
4,STL_CLE_54043,CLE,14820,802,805,805,41.24,-81.51,+5


In [51]:
train_data = pd.merge(trainorigin, Dest_data ,on = 'FlightNumber' )
print(train_data.shape)
train_data.head()

(7861, 29)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y
0,BOS_CLE_29089,2004,9,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14739,BOS,19,30,180,42.22,-71.01,+5,CLE,14820,802,805,805,41.24,-81.51,+5
1,BOS_OAK_55049,2004,9,10,5,820,1200,400,BOS,OAK,2693,10/09/04 12:00,2,14739,BOS,19,30,180,42.22,-71.01,+5,OAK,23230,85,88,88,37.43,-122.13,+8
2,BOS_RSW_47586,2004,3,22,1,700,1036,216,BOS,RSW,1249,22/03/04 10:36,2,14739,BOS,19,30,180,42.22,-71.01,+5,RSW,12894,29,30,32,26.32,-81.45,+5
3,BOS_PIT_49759,2004,5,1,6,845,1031,106,BOS,PIT,496,01/05/04 10:31,2,14739,BOS,19,30,180,42.22,-71.01,+5,PIT,94823,1172,1203,1175,40.30,-80.14,+5
4,BOS_DCA_32791,2004,7,15,4,1300,1430,90,BOS,DCA,399,15/07/04 14:30,2,14739,BOS,19,30,180,42.22,-71.01,+5,DCA,13743,10,65,3,38.52,-77.02,+5


### Creating a **NewID** for merging with the **WeatherData** and **PrecipitationData** 




* The **Year** is the same across the dataset.(2004)


* 6 unique **Months** are present  (['09', '03', '05', '07', '11', '01']) 


* All **Days** are present in a month ([1,2,3, ......, 31])


* **WeatherStationID_Origin** gives the details of the **Departure** place of the flights 


* **WeatherStationID_Destination** gives the details of the **Arrival** place of the flights 



* **NeworiginID** = **WeatherStationID_Origin** + **Month** + **DayofMonth** + **ScheduledDepTime** 

       -> It gives the details of the Origin of the flights [1473909201000]


* **NewDestinationID** = **WeatherStationID_Destination** + **Month** + **DayofMonth** + **ScheduledArrTime** 
   
       -> It gives the details of the Destination of the flights [1482009201154]


In [52]:
train_data['Month'] = train_data['Month'].astype(str)
train_data['DayofMonth'] = train_data['DayofMonth'].astype(str)

train_data['Month'] = train_data['Month'].str.zfill(2)
train_data['DayofMonth'] = train_data['DayofMonth'].str.zfill(2)

train_data['TrainMonyhDay'] = train_data['Month']+train_data['DayofMonth']

train_data.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y,TrainMonyhDay
0,BOS_CLE_29089,2004,09,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14739,BOS,19,30,180,42.22,-71.01,+5,CLE,14820,802,805,805,41.24,-81.51,+5,0920
1,BOS_OAK_55049,2004,09,10,5,820,1200,400,BOS,OAK,2693,10/09/04 12:00,2,14739,BOS,19,30,180,42.22,-71.01,+5,OAK,23230,85,88,88,37.43,-122.13,+8,0910
2,BOS_RSW_47586,2004,03,22,1,700,1036,216,BOS,RSW,1249,22/03/04 10:36,2,14739,BOS,19,30,180,42.22,-71.01,+5,RSW,12894,29,30,32,26.32,-81.45,+5,0322
3,BOS_PIT_49759,2004,05,01,6,845,1031,106,BOS,PIT,496,01/05/04 10:31,2,14739,BOS,19,30,180,42.22,-71.01,+5,PIT,94823,1172,1203,1175,40.30,-80.14,+5,0501
4,BOS_DCA_32791,2004,07,15,4,1300,1430,90,BOS,DCA,399,15/07/04 14:30,2,14739,BOS,19,30,180,42.22,-71.01,+5,DCA,13743,10,65,3,38.52,-77.02,+5,0715


In [53]:
train_data['WeatherStationID_Origin'] = train_data['WeatherStationID_Origin'].astype(str)
train_data['WeatherStationID_Origin'] = train_data['WeatherStationID_Origin'].str.zfill(5)
train_data['WeatherStationID_Destination'] = train_data['WeatherStationID_Destination'].astype(str)
train_data['WeatherStationID_Destination'] = train_data['WeatherStationID_Destination'].str.zfill(5)

train_data['ScheduledDepTime'] = train_data['ScheduledDepTime'].astype(str)
train_data['ScheduledDepTime'] = train_data['ScheduledDepTime'].str.zfill(4)

train_data['ScheduledArrTime'] = train_data['ScheduledArrTime'].astype(str)
train_data['ScheduledArrTime'] = train_data['ScheduledArrTime'].str.zfill(4)

train_data['DayOfWeek'] = train_data['DayOfWeek'].astype(str)

In [54]:
train_data['Origin_NewWeatherStaID'] = train_data['WeatherStationID_Origin'] + train_data['TrainMonyhDay'] + train_data['ScheduledDepTime']
train_data['Destination_NewWeatherStaID'] = train_data['WeatherStationID_Destination'] + train_data['TrainMonyhDay'] + train_data['ScheduledArrTime']


In [55]:
print(train_data.shape)
train_data.head()

(7861, 32)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y,TrainMonyhDay,Origin_NewWeatherStaID,Destination_NewWeatherStaID
0,BOS_CLE_29089,2004,09,20,1,1000,1154,114,BOS,CLE,563,20/09/04 11:54,2,14739,BOS,19,30,180,42.22,-71.01,+5,CLE,14820,802,805,805,41.24,-81.51,+5,0920,1473909201000,1482009201154
1,BOS_OAK_55049,2004,09,10,5,0820,1200,400,BOS,OAK,2693,10/09/04 12:00,2,14739,BOS,19,30,180,42.22,-71.01,+5,OAK,23230,85,88,88,37.43,-122.13,+8,0910,1473909100820,2323009101200
2,BOS_RSW_47586,2004,03,22,1,0700,1036,216,BOS,RSW,1249,22/03/04 10:36,2,14739,BOS,19,30,180,42.22,-71.01,+5,RSW,12894,29,30,32,26.32,-81.45,+5,0322,1473903220700,1289403221036
3,BOS_PIT_49759,2004,05,01,6,0845,1031,106,BOS,PIT,496,01/05/04 10:31,2,14739,BOS,19,30,180,42.22,-71.01,+5,PIT,94823,1172,1203,1175,40.30,-80.14,+5,0501,1473905010845,9482305011031
4,BOS_DCA_32791,2004,07,15,4,1300,1430,90,BOS,DCA,399,15/07/04 14:30,2,14739,BOS,19,30,180,42.22,-71.01,+5,DCA,13743,10,65,3,38.52,-77.02,+5,0715,1473907151300,1374307151430


In [56]:
#train01 = train_data[train_data['Month'] == '01']
#train03 = train_data[train_data['Month'] == '03']
#train05 = trainorigin[trainorigin['Month'] == '05']
#train07 = train_data[train_data['Month'] == '07']
#train09 = trainorigin[trainorigin['Month'] == '09']
#train11 = trainorigin[trainorigin['Month'] == '11']

### 2. Weather information

In [57]:
weather1 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200401hourly.txt')
weather3 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200403hourly.txt')
weather5 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200405hourly.txt')
weather7 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200407hourly.txt')
weather9 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200409hourly.txt')
weather11 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200411hourly.txt')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 3. Precipitation information

In [59]:
precipitation1 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200401hpd.txt')
precipitation3 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200403hpd.txt')
precipitation5 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200405hpd.txt')
precipitation7 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200407hpd.txt')
precipitation9 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200409hpd.txt')
precipitation11 = pd.read_csv('/content/drive/MyDrive/Final hackathon/200411hpd.txt')

* axis : {0/'index', 1/'columns'}, default 0
* The axis to concatenate along.

In [60]:
weather = pd.concat ([weather1, weather3, weather5, weather7, weather9, weather11], axis = 0)

In [61]:
print(weather.shape)
weather.head()

(6485177, 12)


,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure
0,3034,20040101,10,OVC095,10SM,30.0,7.0,38.0,0.0,000,0.0,30.10
1,3035,20040101,10,SCT110,10SM,43.0,18.0,37.0,0.0,000,0.0,30.07
2,3039,20040101,10,OVC002,0.25SM,14.0,12.0,92.0,0.0,000,0.0,30.01
3,3040,20040101,10,CLR,10SM,30.0,21.0,69.0,9.0,250,0.0,29.82
4,4128,20040101,10,BKN001 BKN010 OVC024,4SM,NaN,NaN,NaN,0.0,000,0.0,29.80


In [62]:
precipitation = pd.concat([precipitation1, precipitation3, precipitation5, precipitation7, precipitation9, precipitation11], axis = 0)

In [63]:
print(precipitation.shape)
precipitation.head()

(7697386, 4)


,WeatherStationID,YearMonthDay,Time,HourlyPrecip
0,3034,20040101,10,0.0
1,3035,20040101,10,0.0
2,3039,20040101,10,0.0
3,3040,20040101,10,0.0
4,4128,20040101,10,0.0


* groupby **Origin_NewWeatherStaID** and sorting the rows (ascending=**True**)

In [98]:
origin_data = train_data.groupby('Origin_NewWeatherStaID').head().sort_values('Origin_NewWeatherStaID').reset_index()
origin_data.drop('index', axis = 1, inplace = True)
print(origin_data.shape)
origin_data.head()

(7861, 32)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y,TrainMonyhDay,Origin_NewWeatherStaID,Destination_NewWeatherStaID
0,DEN_LAS_43741,2004,01,01,4,0815,0902,107,DEN,LAS,629,01/01/04 09:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,LAS,23169,2127,2180,2091,36.05,-115.10,+8,0101,301701010815,2316901010902
1,DEN_GJT_26718,2004,01,01,4,1115,1223,68,DEN,GJT,212,01/01/04 13:35,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,GJT,23066,4823,4839,4826,39.08,-108.32,+7,0101,301701011115,2306601011223
2,DEN_MCO_42612,2004,01,01,4,1245,1802,197,DEN,MCO,1545,01/01/04 18:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,MCO,12815,95,106,98,28.26,-81.19,+5,0101,301701011245,1281501011802
3,DEN_SLC_47047,2004,01,03,6,0610,0735,85,DEN,SLC,391,03/01/04 09:22,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,SLC,24127,4221,4227,4224,40.47,-111.58,+7,0103,301701030610,2412701030735
4,DEN_RAP_26337,2004,01,03,6,1115,1220,65,DEN,RAP,301,03/01/04 14:49,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,RAP,24090,3150,3168,3153,44.03,-103.03,+7,0103,301701031115,2409001031220


In [99]:
col = ['FlightNumber','Destination_NewWeatherStaID','ScheduledArrTime' ]
Dest_train_data =origin_data[col]
Dest_train_data.head()

,FlightNumber,Destination_NewWeatherStaID,ScheduledArrTime
0,DEN_LAS_43741,2316901010902,0902
1,DEN_GJT_26718,2306601011223,1223
2,DEN_MCO_42612,1281501011802,1802
3,DEN_SLC_47047,2412701030735,0735
4,DEN_RAP_26337,2409001031220,1220


In [100]:
destination_data = Dest_train_data.groupby('Destination_NewWeatherStaID').head().sort_values('Destination_NewWeatherStaID').reset_index()

destination_data.drop('index', axis = 1, inplace = True)
print(destination_data.shape)
destination_data.head()

(7861, 3)


,FlightNumber,Destination_NewWeatherStaID,ScheduledArrTime
0,TUL_DEN_37995,0301701021620,1620
1,DFW_DEN_51239,0301701031250,1250
2,ELP_DEN_43943,0301701031853,1853
3,LNK_DEN_44116,0301701041030,1030
4,DFW_DEN_37437,0301701041046,1046


In [67]:
weather.dtypes

WeatherStationID             int64
YearMonthDay                 int64
Time                         int64
SkyConditions               object
Visibility                  object
DBT                        float64
DewPointTemp               float64
RelativeHumidityPercent    float64
WindSpeed                   object
WindDirection               object
WindGustValue              float64
StationPressure            float64
dtype: object

In [68]:
precipitation.dtypes

WeatherStationID      int64
YearMonthDay          int64
Time                  int64
HourlyPrecip        float64
dtype: object

# SkyConditions - Clouds
The cloud cover is listed per layer. Per layer you see:

The cover in octals (1/8 parts):
 * FEW few clouds: 1/8th or 2/8th cover
 * SCT scattered clouds: 3/8th or 4/8th cover
 * BKN broken clouds: 5/8th to 7/8th cover
 * OVC overcast clouds: 8/8th cover 

 The height of the bottom of the cloud (cloud base) in hundreds of feet above the airport: FEW007 means a cloud base of 700 ft above site height.

 ### Special codes:
* NSC No significant cloud cover. This means that there is no cloud below the 5,000 ft, but there is above it. That cloud cover is not cumulonimbus or towering cumulus.
* SKC No cloud cover (Determined by meteorologist)
* NCD No clouds measured (Automatic weather stations)
* CLR No cloud cover detected below the 12,000 ft (Automatic weather stations)
* OVC overcast clouds: 8/8th cover

 #### Example

FEW007 BKN014CB BKN017 means three layers of cloud:

* Few clouds at an altitude of 700 ft
*  Broken clouds at an altitude of 1,400 ft, with cumulonimbus clouds
* Broken clouds at an altitude of 1,700 ft
* OVC002 is overcast at 200ft- hence the low visibility.

source : [Link](https://metar-taf.com/explanation)

# visibility 

* The visibility is the measure of the distance at which an object or light can be clearly discerned.Visibility is affected by the presence of fog, cloud, haze and precipitation.

* Visibility is an important factor in all phases of flight, but especially when the aircraft is manoeuvring on or close to the ground, i.e. during taxi-out, take-off and initial climb, approach and landing, and taxi-in.

* In very low visibility, it may prove impossible for the pilot to navigate the aircraft along the runway and taxiways to the aircraft stand.

* Visibility is measured in metres (or kilometres) or in Statute Miles, depending on the country concerned.

* 3SM means the visibility is 3 miles/SM (1SM = 1.6Km) (statute mile). 3/4SM means visibility is 0.75 miles.

source : [Link](https://www.skybrary.aero/articles/visibility#:~:text=Aircraft%20departure%20and%20arrival%20is,qualification%20of%20the%20flight%20crew.)

# Dry-bulb temperature

The dry-bulb temperature (DBT) is the temperature of air measured by a thermometer freely exposed to the air, but shielded from radiation and moisture. DBT is the temperature that is usually thought of as air temperature, and it is the true thermodynamic temperature. It is directly proportional to the mean kinetic energy of the air molecules. Temperature is usually measured in degrees Celsius (°C), kelvins (K), or degrees Fahrenheit (°F).

# Dew point

The dew point is the temperature to which air must be cooled to become saturated with water vapor, assuming constant air pressure and water content. When cooled below the dew point, moisture capacity is reduced and airborne water vapor will condense to form liquid water known as dew. When this occurs via contact with a colder surface, dew will form on that surface.

The dew point is affected by humidity. When there is more moisture in the air, the dew point is higher.

# RelativeHumidityPercent

* it is a measure of the actual amount of water vapor in the air compared to the total amount of vapor that can exist in the air at its current temperature.

* Relative humidity, often expressed as a percentage

 # Wind Speed

wind speed, or wind flow speed, is a fundamental atmospheric quantity caused by air moving from high to low pressure, usually due to changes in temperature.

* Wind speed is typically reported in miles per hour, knots, or meters per second. One mile per hour is equal to 0.45 meters per second, and 0.87 knots. (1 knots = 1.852 Kmph )

# Wind direction

Wind direction is generally reported by the direction from which it originates. For example, a north or northerly wind blows from the north to the south.

* Wind direction is usually reported in cardinal (or compass) direction, or in degrees. Consequently, a wind blowing from the north has a wind direction referred to as 0° (360°); a wind blowing from the east has a wind direction referred to as 90°, etc.

# WindGustValue 


 WindGust is a sudden, brief increase in speed of the wind. According to U.S. weather observing practice, gusts are reported when the peak wind speed reaches at least 16 knots and the variation in wind speed between the peaks and lulls is at least 9 knots. The duration of a gust is usually less than 20 seconds.

* The gust will last generally under 20 seconds. The longer that gust lasts, the longer that gust can exert force on an object and the higher the potential to cause damage. 

# StationPressure

This is the pressure that is observed at a specific elevation and is the true barometric pressure of a location. It is the pressure exerted by the atmosphere at a point as a result of gravity acting upon the "column" of air that lies directly above the point. Consequently, higher elevations above sea level experience lower pressure since there is less atmosphere on which gravity can act. Put another way, the weight of the atmosphere decreases as one increases in elevation. Consequently then, in general, for every thousand feet of elevation gain, the pressure drops about 1 inch of mercury. 

# Precipitation 

precipitation is any product of the condensation of atmospheric water vapor that falls under gravitational pull from clouds.The main forms of precipitation include drizzle, rain, sleet, snow, ice pellets, graupel and hail. Precipitation occurs when a portion of the atmosphere becomes saturated with water vapor (reaching 100% relative humidity), so that the water condenses and "precipitates" or falls

In [69]:
weather['SkyConditions'].unique()

array(['OVC095', 'SCT110', 'OVC002', ..., 'SCT005 BKN018 BKN040',
       'FEW000 BKN035 OVC050', 'SCT024 SCT050 SCT100'], dtype=object)

### Converting Wind Speed from object to float datadypte

In [71]:
weather['WindSpeed'].unique()

array([0.0, 9.0, 4.0, 7.0, 10.0, 5.0, 6.0, 11.0, 13.0, 14.0, 3.0, 23.0,
       17.0, 1.0, 15.0, 24.0, 31.0, 8.0, 29.0, 12.0, 22.0, 18.0, 21.0,
       25.0, 16.0, 26.0, 28.0, nan, 20.0, 27.0, 19.0, 32.0, 34.0, 30.0,
       2.0, 36.0, 35.0, 33.0, 39.0, 40.0, 37.0, 38.0, 43.0, 41.0, 44.0,
       45.0, 42.0, 50.0, 47.0, 48.0, 49.0, 51.0, '5', '11', '4', '3', '6',
       '16', '8', '15', '0', '7', '29', '9', '12', '14', '10', '13', '20',
       '22', '17', '18', '23', '26', '21', '19', '30', '25', '28', '24',
       '27', '31', '35', '36', '34', '2', '41', '1', '33', '32', '39',
       '37', '42', '38', '40', '45', '46', '/1', 46.0, 71.0, 53.0, 52.0,
       73.0, '44', '47', '48', '51', '52', '43', '/0', '/2', '96', 55.0,
       54.0, 96.0, 56.0, 72.0, 95.0, '57', 61.0, 58.0, 57.0, 59.0, 65.0,
       '50', '49', '53', '-3', 78.0, 98.0], dtype=object)

In [72]:
c = weather['WindSpeed'].value_counts()
c.loc['/1']

53

In [73]:
weather[(weather['WindSpeed'] == '/0') | (weather['WindSpeed'] == '/1')  | (weather['WindSpeed'] == '/2') ].head()

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure
256622,13882,20040309,1153,SCT060 BKN085 BKN100,10SM,56.0,30.0,37.0,/1,290,19.0,29.15
614990,24157,20040511,856,FEW030 BKN060 OVC095,10SM,NaN,NaN,NaN,/1,030,0.0,27.28
630223,93230,20040512,1453,CLR,10SM,58.0,26.0,30.0,/0,170,0.0,23.78
694880,41606,20040513,1614,CLR,10SM,NaN,NaN,NaN,/1,070,0.0,29.86
697183,41606,20040513,1703,FEW020 SCT026,10SM,NaN,NaN,NaN,/1,070,0.0,29.87


Replacing ['/0', '/1', '/2'] by  ['0', '1', '2'] in the windspeed column to change it into **float datatype** 

In [74]:
weather.WindSpeed.replace(['/0', '/1', '/2'], ['0', '1', '2'], inplace=True)

In [75]:
weather1['WindSpeed'] = weather1['WindSpeed'].astype(str).astype(float)

### Creating a New ID in the weather dataset similar to the Train ID 

* **New ID** = **WeatherStationID** + **Weather_MonthDay** + **Time**

In [78]:
weather['YearMonthDay'] = weather['YearMonthDay'].astype(str)

weatherMD = weather['YearMonthDay'].values
whe_Month_day = []
for vals in weatherMD :
    md = vals[4:]
    whe_Month_day.append(md)
    
weather['Weather_MonthDay'] = whe_Month_day
weather.head()

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,Weather_MonthDay
0,3034,20040101,10,OVC095,10SM,30.0,7.0,38.0,0.0,000,0.0,30.10,0101
1,3035,20040101,10,SCT110,10SM,43.0,18.0,37.0,0.0,000,0.0,30.07,0101
2,3039,20040101,10,OVC002,0.25SM,14.0,12.0,92.0,0.0,000,0.0,30.01,0101
3,3040,20040101,10,CLR,10SM,30.0,21.0,69.0,9.0,250,0.0,29.82,0101
4,4128,20040101,10,BKN001 BKN010 OVC024,4SM,NaN,NaN,NaN,0.0,000,0.0,29.80,0101


In [79]:
weather['WeatherStationID'] = weather['WeatherStationID'].astype(str)
weather['WeatherStationID'] = weather['WeatherStationID'].str.zfill(5)
weather['Time'] = weather['Time'].astype(str)
weather['Time'] = weather['Time'].str.zfill(4)

weather['New_WeatherStationID'] = weather['WeatherStationID'] + weather['Weather_MonthDay'] + weather['Time'] 


In [ ]:
weather1.head()

,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,Weather_MonthDay,New_WeatherStationID
0,03034,20040101,0010,OVC095,10SM,30.0,7.0,38.0,0.0,000,0.0,30.10,0101,0303401010010
1,03035,20040101,0010,SCT110,10SM,43.0,18.0,37.0,0.0,000,0.0,30.07,0101,0303501010010
2,03039,20040101,0010,OVC002,0.25SM,14.0,12.0,92.0,0.0,000,0.0,30.01,0101,0303901010010
3,03040,20040101,0010,CLR,10SM,30.0,21.0,69.0,9.0,250,0.0,29.82,0101,0304001010010
4,04128,20040101,0010,BKN001 BKN010 OVC024,4SM,NaN,NaN,NaN,0.0,000,0.0,29.80,0101,0412801010010


In [80]:
weather_data = weather.groupby('New_WeatherStationID').head().sort_values('New_WeatherStationID').reset_index()
weather_data.drop('index', axis = 1, inplace = True)
print(weather_data.shape)

weather_data.head()

(6485177, 14)


,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,Weather_MonthDay,New_WeatherStationID
0,03011,20040101,1350,FEW075 BKN090,10SM,32.0,10.0,40.0,15.0,160,0.0,30.05,0101,0301101011350
1,03011,20040101,1910,BKN028 OVC033,10SM,21.0,14.0,74.0,5.0,VRB,0.0,30.06,0101,0301101011910
2,03011,20040101,2010,FEW025 OVC032,9SM,25.0,18.0,75.0,4.0,VRB,0.0,30.06,0101,0301101012010
3,03011,20040102,0410,FEW045 BKN060 OVC080,10SM,34.0,19.0,54.0,16.0,240,27.0,29.92,0102,0301101020410
4,03011,20040102,0730,BKN010 OVC017,1.75SM,25.0,21.0,85.0,0.0,000,0.0,29.95,0102,0301101020730


In [82]:
null = weather_data.isnull().sum()
null[null>0]

SkyConditions               162260
Visibility                  151688
DBT                        1162698
DewPointTemp               1176789
RelativeHumidityPercent    1179911
WindSpeed                    33407
WindDirection                33392
WindGustValue                43844
StationPressure             805551
dtype: int64

## Imputing Null values 

* Replace NULL values with the value from the previous row. 

* The **ffill()** method replaces the NULL values with the value from the **previous row** (or previous column, if the axis parameter is set to 'columns').


In [83]:
weather_data['SkyConditions']= weather_data['SkyConditions'].fillna(method ='ffill')

In [84]:
weather_data['Visibility']= weather_data['Visibility'].fillna(method ='ffill')


In [85]:
weather_data['DBT']= weather_data['DBT'].fillna(method ='ffill')


In [86]:
weather_data['DewPointTemp']= weather_data['DewPointTemp'].fillna(method ='ffill')


In [87]:
weather_data['RelativeHumidityPercent']= weather_data['RelativeHumidityPercent'].fillna(method ='ffill')


In [88]:
weather_data['WindSpeed']= weather_data['WindSpeed'].fillna(method ='ffill')


In [89]:
weather_data['WindDirection']= weather_data['WindDirection'].fillna(method ='ffill')


In [90]:
weather_data['WindGustValue']= weather_data['WindGustValue'].fillna(method ='ffill')


In [91]:
weather_data['StationPressure']= weather_data['StationPressure'].fillna(method ='ffill')


In [96]:
a = weather_data.isnull().sum()
print('Number of Null values in the Weather dataset : ', len(a[a>0]))

Number of Null values in the Weather dataset :  0


# Pandas.merge_asof¶


* Perform an asof merge. This is similar to a left-join except that we match on nearest key rather than equal keys.

* Both DataFrames must be sorted by the key. 

* A “backward” search selects the last row in the right DataFrame whose ‘on’ key is less than or equal to the left’s key.
* A “forward” search selects the first row in the right DataFrame whose ‘on’ key is greater than or equal to the left’s key.
* A “nearest” search selects the row in the right DataFrame whose ‘on’ key is closest in absolute distance to the left’s key.

Source1 : [Pandas](https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.merge_asof.html)

Source2 : [Link](https://blog.finxter.com/pandas-merge_asof-a-simple-guide-with-video/)

Source3 : [Youtube](https://www.youtube.com/watch?v=CgH6-mNnQrA&t=39s)

In [103]:
origin_data['Origin_NewWeatherStaID'] = origin_data['Origin_NewWeatherStaID'] .astype(str).astype(int)
weather_data['New_WeatherStationID'] = weather_data['New_WeatherStationID'] .astype(str).astype(int)
destination_data['Destination_NewWeatherStaID'] = destination_data['Destination_NewWeatherStaID'].astype(str).astype(int)

## for example 

*  **ScheduledDepTime** = 0815, 1115, 1245
 
 
*  **Origin_NewWeatherStaID** = 301701010815, 301701011115, 301701011245, 


*  **New_WeatherStationID** = 301701010853, 301701011153, 301701011253


## Flight Departure Weather Information.

In [101]:
train_origin  =  pd.merge_asof(origin_data, weather_data, left_on='Origin_NewWeatherStaID',right_on = 'New_WeatherStationID', direction = 'nearest')


In [ ]:
train_origin.head()

,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y,TrainMonyhDay,Origin_NewWeatherStaID,Destination_NewWeatherStaID,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,Weather_MonthDay,New_WeatherStationID
0,DEN_LAS_43741,2004,01,01,4,0815,0902,107,DEN,LAS,629,01/01/04 09:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,LAS,23169,2127,2180,2091,36.05,-115.10,+8,0101,301701010815,2316901010902,03017,20040101,0853,FEW100 SCT150,10SM,37.0,12.0,36.0,6.0,170,0.0,24.55,0101,301701010853
1,DEN_GJT_26718,2004,01,01,4,1115,1223,68,DEN,GJT,212,01/01/04 13:35,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,GJT,23066,4823,4839,4826,39.08,-108.32,+7,0101,301701011115,2306601011223,03017,20040101,1153,FEW090 SCT140 BKN220,10SM,50.0,7.0,17.0,6.0,190,0.0,24.49,0101,301701011153
2,DEN_MCO_42612,2004,01,01,4,1245,1802,197,DEN,MCO,1545,01/01/04 18:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,MCO,12815,95,106,98,28.26,-81.19,+5,0101,301701011245,1281501011802,03017,20040101,1253,FEW090 SCT140 BKN200,10SM,50.0,8.0,18.0,8.0,100,0.0,24.45,0101,301701011253
3,DEN_SLC_47047,2004,01,03,6,0610,0735,85,DEN,SLC,391,03/01/04 09:22,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,SLC,24127,4221,4227,4224,40.47,-111.58,+7,0103,301701030610,2412701030735,03017,20040103,0619,BKN020 BKN034 OVC049,2SM,23.0,22.0,96.0,11.0,120,0.0,24.45,0103,301701030619
4,DEN_RAP_26337,2004,01,03,6,1115,1220,65,DEN,RAP,301,03/01/04 14:49,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,RAP,24090,3150,3168,3153,44.03,-103.03,+7,0103,301701031115,2409001031220,03017,20040103,1153,SCT009 BKN015 OVC026,1SM,24.0,20.0,84.0,14.0,140,0.0,24.40,0103,301701031153


## Flight Arrival Weather Information. 


## for example


*  **ScheduledArrTime** = 1620, 1250, 1853


*  **Destination_NewWeatherStaID** = 301701021620, 301701031250, 301701031853


*  **New_WeatherStationID**  = 301701021653, 301701031253, 301701031853

In [105]:
train_dest  =  pd.merge_asof(destination_data, weather_data, left_on='Destination_NewWeatherStaID',right_on = 'New_WeatherStationID', direction = 'nearest')


In [106]:
train_dest.head()

,FlightNumber,Destination_NewWeatherStaID,ScheduledArrTime,WeatherStationID,YearMonthDay,Time,SkyConditions,Visibility,DBT,DewPointTemp,RelativeHumidityPercent,WindSpeed,WindDirection,WindGustValue,StationPressure,Weather_MonthDay,New_WeatherStationID
0,TUL_DEN_37995,301701021620,1620,03017,20040102,1653,FEW030 SCT075 BKN130 BKN200,10SM,33.0,17.0,52.0,12.0,040,19.0,24.44,0102,301701021653
1,DFW_DEN_51239,301701031250,1250,03017,20040103,1253,FEW010 BKN014 BKN060,10SM,26.0,20.0,78.0,17.0,120,20.0,24.37,0103,301701031253
2,ELP_DEN_43943,301701031853,1853,03017,20040103,1853,FEW060 SCT095 OVC140,10SM,23.0,21.0,92.0,11.0,140,0.0,24.37,0103,301701031853
3,LNK_DEN_44116,301701041030,1030,03017,20040104,1027,SCT005 SCT010,7SM,19.0,18.0,96.0,5.0,150,0.0,24.42,0104,301701041027
4,DFW_DEN_37437,301701041046,1046,03017,20040104,1053,FEW010 SCT070,10SM,20.0,15.0,81.0,4.0,160,0.0,24.63,0104,301701041053


## Joining Departure and Arrival weather information.

In [107]:
train_full = pd.merge(train_origin,train_dest , on = 'FlightNumber' )

In [108]:
print(train_full.shape)
train_full.head()

(7861, 62)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime_x,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y,TrainMonyhDay,Origin_NewWeatherStaID,Destination_NewWeatherStaID_x,WeatherStationID_x,YearMonthDay_x,Time_x,SkyConditions_x,Visibility_x,DBT_x,DewPointTemp_x,RelativeHumidityPercent_x,WindSpeed_x,WindDirection_x,WindGustValue_x,StationPressure_x,Weather_MonthDay_x,New_WeatherStationID_x,Destination_NewWeatherStaID_y,ScheduledArrTime_y,WeatherStationID_y,YearMonthDay_y,Time_y,SkyConditions_y,Visibility_y,DBT_y,DewPointTemp_y,RelativeHumidityPercent_y,WindSpeed_y,WindDirection_y,WindGustValue_y,StationPressure_y,Weather_MonthDay_y,New_WeatherStationID_y
0,DEN_LAS_43741,2004,01,01,4,0815,0902,107,DEN,LAS,629,01/01/04 09:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,LAS,23169,2127,2180,2091,36.05,-115.10,+8,0101,301701010815,2316901010902,03017,20040101,0853,FEW100 SCT150,10SM,37.0,12.0,36.0,6.0,170,0.0,24.55,0101,301701010853,2316901010902,0902,23169,20040101,0856,BKN140 OVC200,10SM,47.0,24.0,41.0,11.0,170,0.0,27.86,0101,2316901010856
1,DEN_GJT_26718,2004,01,01,4,1115,1223,68,DEN,GJT,212,01/01/04 13:35,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,GJT,23066,4823,4839,4826,39.08,-108.32,+7,0101,301701011115,2306601011223,03017,20040101,1153,FEW090 SCT140 BKN220,10SM,50.0,7.0,17.0,6.0,190,0.0,24.49,0101,301701011153,2306601011223,1223,23066,20040101,1253,OVC095,8SM,36.0,31.0,82.0,8.0,290,0.0,25.14,0101,2306601011253
2,DEN_MCO_42612,2004,01,01,4,1245,1802,197,DEN,MCO,1545,01/01/04 18:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,MCO,12815,95,106,98,28.26,-81.19,+5,0101,301701011245,1281501011802,03017,20040101,1253,FEW090 SCT140 BKN200,10SM,50.0,8.0,18.0,8.0,100,0.0,24.45,0101,301701011253,1281501011802,1802,12815,20040101,1753,FEW038,10SM,69.0,63.0,81.0,11.0,080,0.0,30.24,0101,1281501011753
3,DEN_SLC_47047,2004,01,03,6,0610,0735,85,DEN,SLC,391,03/01/04 09:22,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,SLC,24127,4221,4227,4224,40.47,-111.58,+7,0103,301701030610,2412701030735,03017,20040103,0619,BKN020 BKN034 OVC049,2SM,23.0,22.0,96.0,11.0,120,0.0,24.45,0103,301701030619,2412701030735,0735,24127,20040103,0756,BKN031 BKN044 OVC055,9SM,24.0,18.0,77.0,9.0,330,0.0,25.50,0103,2412701030756
4,DEN_RAP_26337,2004,01,03,6,1115,1220,65,DEN,RAP,301,03/01/04 14:49,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,RAP,24090,3150,3168,3153,44.03,-103.03,+7,0103,301701031115,2409001031220,03017,20040103,1153,SCT009 BKN015 OVC026,1SM,24.0,20.0,84.0,14.0,140,0.0,24.40,0103,301701031153,2409001031220,1220,24090,20040103,1221,FEW010 BKN014 OVC033,8SM,9.0,4.0,80.0,5.0,110,0.0,26.72,0103,2409001031221


In [109]:
train_full.to_csv('TRAINDATA AND EWATHERDATA SET')

# Precipitation Data

In [110]:
print(precipitation.shape)
precipitation.head()

(7697386, 4)


,WeatherStationID,YearMonthDay,Time,HourlyPrecip
0,3034,20040101,10,0.0
1,3035,20040101,10,0.0
2,3039,20040101,10,0.0
3,3040,20040101,10,0.0
4,4128,20040101,10,0.0


In [111]:
precipitation['YearMonthDay'] =  precipitation['YearMonthDay'].astype(str)
precipitation['WeatherStationID'] =  precipitation['WeatherStationID'].astype(str)
precipitation['WeatherStationID'] = precipitation['WeatherStationID'].str.zfill(5)
precipitation['Time'] =  precipitation['Time'].astype(str)
precipitation['Time'] = precipitation['Time'].str.zfill(4)


In [112]:
prepmd = precipitation['YearMonthDay'].values
prep_monthday = []
for values in prepmd :
    pmd = values[4:]
    prep_monthday.append(pmd)

precipitation['Prept_MonthDay'] = prep_monthday

In [113]:
precipitation.dtypes

WeatherStationID     object
YearMonthDay         object
Time                 object
HourlyPrecip        float64
Prept_MonthDay       object
dtype: object

In [114]:
precipitation['Prept_New_Weatherstat_ID'] = precipitation['WeatherStationID'] + precipitation['Prept_MonthDay']+precipitation['Time']
print(precipitation.shape)
precipitation.head()

(7697386, 6)


,WeatherStationID,YearMonthDay,Time,HourlyPrecip,Prept_MonthDay,Prept_New_Weatherstat_ID
0,03034,20040101,0010,0.0,0101,0303401010010
1,03035,20040101,0010,0.0,0101,0303501010010
2,03039,20040101,0010,0.0,0101,0303901010010
3,03040,20040101,0010,0.0,0101,0304001010010
4,04128,20040101,0010,0.0,0101,0412801010010


In [ ]:
precipitation.to_csv('TRAIN PRECIPITATION DATASET')

In [115]:
v = precipitation.isnull().sum()
v[v>0]

HourlyPrecip    251266
dtype: int64

In [116]:
precipitation_data1 = precipitation.groupby('Prept_New_Weatherstat_ID').head().sort_values('Prept_New_Weatherstat_ID').reset_index()
precipitation_data1.drop('index', axis = 1, inplace = True)


### Imputing Null values using ffill()

In [117]:
precipitation_data1['HourlyPrecip']= precipitation_data1['HourlyPrecip'].fillna(method ='ffill')


In [118]:
null = precipitation_data1.isnull().sum()
print('Total number of null values in the precipitation dataset : ', len(null[null>0]))

Total number of null values in the precipitation dataset :  0


In [119]:
col = ['Prept_New_Weatherstat_ID', 'HourlyPrecip', 'Time' ]
prep_datanew2 = precipitation_data1[col]
prep_datanew2.head()

,Prept_New_Weatherstat_ID,HourlyPrecip,Time
0,0301101011350,0.0,1350
1,0301101011910,0.0,1910
2,0301101012010,0.0,2010
3,0301101020410,0.0,0410
4,0301101020730,0.0,0730


In [120]:
prep_datanew2['Prept_New_Weatherstat_ID'] = prep_datanew2['Prept_New_Weatherstat_ID'].astype(str).astype(int)

<ipython-input-120-5ff8d86743a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_datanew2['Prept_New_Weatherstat_ID'] = prep_datanew2['Prept_New_Weatherstat_ID'].astype(str).astype(int)


In [121]:
train_o =  pd.merge_asof(train_full,prep_datanew2, left_on = 'Origin_NewWeatherStaID', right_on = 'Prept_New_Weatherstat_ID', direction = 'nearest')

In [122]:
train_d  =  pd.merge_asof(destination_data, prep_datanew2, left_on='Destination_NewWeatherStaID',right_on = 'Prept_New_Weatherstat_ID', direction = 'nearest')


In [123]:
train_d.head()

,FlightNumber,Destination_NewWeatherStaID,ScheduledArrTime,Prept_New_Weatherstat_ID,HourlyPrecip,Time
0,TUL_DEN_37995,301701021620,1620,301701021653,0.00,1653
1,DFW_DEN_51239,301701031250,1250,301701031253,0.01,1253
2,ELP_DEN_43943,301701031853,1853,301701031853,0.00,1853
3,LNK_DEN_44116,301701041030,1030,301701041053,0.00,1053
4,DFW_DEN_37437,301701041046,1046,301701041053,0.00,1053


In [124]:
full_train = pd.merge(train_o,train_d , on = 'FlightNumber' )

<ipython-input-124-2817e1ec61a5>:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Time_x'} in the result is deprecated and will raise a MergeError in a future version.
  full_train = pd.merge(train_o,train_d , on = 'FlightNumber' )


In [125]:
print(full_train.shape)
full_train.head()

(7861, 70)


,FlightNumber,Year,Month,DayofMonth,DayOfWeek,ScheduledDepTime,ScheduledArrTime_x,ScheduledTravelTime,Origin,Destination,Distance,ActualArrivalTimeStamp,FlightDelayStatus,WeatherStationID_Origin,AirportID_x,GroundHeight_x,StationHeight_x,BarometerHeight_x,Latitude_x,Longitude_x,TimeZone_x,AirportID_y,WeatherStationID_Destination,GroundHeight_y,StationHeight_y,BarometerHeight_y,Latitude_y,Longitude_y,TimeZone_y,TrainMonyhDay,Origin_NewWeatherStaID,Destination_NewWeatherStaID_x,WeatherStationID_x,YearMonthDay_x,Time_x,SkyConditions_x,Visibility_x,DBT_x,DewPointTemp_x,RelativeHumidityPercent_x,WindSpeed_x,WindDirection_x,WindGustValue_x,StationPressure_x,Weather_MonthDay_x,New_WeatherStationID_x,Destination_NewWeatherStaID_y,ScheduledArrTime_y,WeatherStationID_y,YearMonthDay_y,Time_y,SkyConditions_y,Visibility_y,DBT_y,DewPointTemp_y,RelativeHumidityPercent_y,WindSpeed_y,WindDirection_y,WindGustValue_y,StationPressure_y,Weather_MonthDay_y,New_WeatherStationID_y,Prept_New_Weatherstat_ID_x,HourlyPrecip_x,Time_x,Destination_NewWeatherStaID,ScheduledArrTime,Prept_New_Weatherstat_ID_y,HourlyPrecip_y,Time_y
0,DEN_LAS_43741,2004,01,01,4,0815,0902,107,DEN,LAS,629,01/01/04 09:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,LAS,23169,2127,2180,2091,36.05,-115.10,+8,0101,301701010815,2316901010902,03017,20040101,0853,FEW100 SCT150,10SM,37.0,12.0,36.0,6.0,170,0.0,24.55,0101,301701010853,2316901010902,0902,23169,20040101,0856,BKN140 OVC200,10SM,47.0,24.0,41.0,11.0,170,0.0,27.86,0101,2316901010856,301701010853,0.00,0853,2316901010902,0902,2316901010856,0.0,0856
1,DEN_GJT_26718,2004,01,01,4,1115,1223,68,DEN,GJT,212,01/01/04 13:35,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,GJT,23066,4823,4839,4826,39.08,-108.32,+7,0101,301701011115,2306601011223,03017,20040101,1153,FEW090 SCT140 BKN220,10SM,50.0,7.0,17.0,6.0,190,0.0,24.49,0101,301701011153,2306601011223,1223,23066,20040101,1253,OVC095,8SM,36.0,31.0,82.0,8.0,290,0.0,25.14,0101,2306601011253,301701011153,0.00,1153,2306601011223,1223,2306601011253,0.0,1253
2,DEN_MCO_42612,2004,01,01,4,1245,1802,197,DEN,MCO,1545,01/01/04 18:02,2,03017,DEN,5379,5431,5382,39.5,-104.4,+7,MCO,12815,95,106,98,28.26,-81.19,+5,0101,301701011245,1281501011802,03017,20040101,1253,FEW090 SCT140 BKN200,10SM,50.0,8.0,18.0,8.0,100,0.0,24.45,0101,301701011253,1281501011802,1802,12815,20040101,1753,FEW038,10SM,69.0,63.0,81.0,11.0,080,0.0,30.24,0101,1281501011753,301701011253,0.00,1253,1281501011802,1802,1281501011753,0.0,1753
3,DEN_SLC_47047,2004,01,03,6,0610,0735,85,DEN,SLC,391,03/01/04 09:22,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,SLC,24127,4221,4227,4224,40.47,-111.58,+7,0103,301701030610,2412701030735,03017,20040103,0619,BKN020 BKN034 OVC049,2SM,23.0,22.0,96.0,11.0,120,0.0,24.45,0103,301701030619,2412701030735,0735,24127,20040103,0756,BKN031 BKN044 OVC055,9SM,24.0,18.0,77.0,9.0,330,0.0,25.50,0103,2412701030756,301701030653,0.01,0653,2412701030735,0735,2412701030756,0.0,0756
4,DEN_RAP_26337,2004,01,03,6,1115,1220,65,DEN,RAP,301,03/01/04 14:49,1,03017,DEN,5379,5431,5382,39.5,-104.4,+7,RAP,24090,3150,3168,3153,44.03,-103.03,+7,0103,301701031115,2409001031220,03017,20040103,1153,SCT009 BKN015 OVC026,1SM,24.0,20.0,84.0,14.0,140,0.0,24.40,0103,301701031153,2409001031220,1220,24090,20040103,1221,FEW010 BKN014 OVC033,8SM,9.0,4.0,80.0,5.0,110,0.0,26.72,0103,2409001031221,301701031153,0.01,1153,2409001031220,1220,2409001031252,0.0,1252


In [126]:
full_train.to_csv('TRAIN WEATHER PRECIPITATION')